In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/senior-director-applied-science-remote-work-option/job/R-45010")
page_data = loader.load().pop().page_content
print(page_data)























Senior Director,  Applied Science (Remote Work Option)










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to 

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [12]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Director, Applied Science',
 'experience': '10 years of experience leading multiple science and engineering teams, at least 15 years of experience in building and applying ML models',
 'skills': ['Computer science',
  'Statistics',
  'Mathematics',
  'Software engineering',
  'Machine learning',
  'Generative AI',
  'Large language models',
  'Reinforcement learning',
  'Supervised learning',
  'Programming languages'],
 'description': "We are seeking a visionary leader to spearhead CPI's Advanced Machine Learning initiatives, aimed at revolutionizing Nike's product creation lifecycle—from ideation through to supply chain. This role requires a deep learning expert who is well-versed in generative AI advancements and capable of guiding cutting-edge research in these domains."}

In [15]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [16]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [18]:
job=json_res


In [20]:
job['skills']

['Computer science',
 'Statistics',
 'Mathematics',
 'Software engineering',
 'Machine learning',
 'Generative AI',
 'Large language models',
 'Reinforcement learning',
 'Supervised learning',
 'Programming languages']

In [21]:
links=collection.query(query_texts=job['skills'],n_results=2).get('metadatas',[])

In [22]:
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links':

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Revolutionizing Nike's Product Creation Lifecycle with AtliQ's AI Expertise

Dear Hiring Manager,

I came across the job posting for Senior Director, Applied Science at Nike, and I was impressed by the company's vision to revolutionize its product creation lifecycle using Advanced Machine Learning initiatives. As a business development executive at AtliQ, an AI & Software Consulting company, I believe our team can help fulfill Nike's needs in this domain.

At AtliQ, we have a proven track record of empowering enterprises with tailored AI and software solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in Machine Learning, Generative AI, and Large Language Models aligns perfectly with Nike's requirements. We have successfully developed and implemented ML models using Python, and our portfolio showcases our capabilities in this area: https://example.com/ml-python-portfolio.

Our team of experts has extensive ex